In [2]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    "https://f0c40d3bde77471f83e6cace87537665.ap-southeast-1.aws.found.io",
    api_key="MU9oZlg1Y0JiN3JESEE4V0Y5aUg6N1lNZ0VaREtPU1pob3hYQWQtNU1zZw=="
)

index_name = "cve-index"

# Delete if exists (optional cleanup step for testing)
if client.indices.exists(index=index_name):
    client.indices.delete(index=index_name)

# Create index with proper mapping
mappings = {
    "mappings": {
        "properties": {
            "text": {
                "properties": {
                    "cve_id": {"type": "keyword"},
                    "description": {"type": "text"},
                    "severity": {"type": "keyword"}
                }
            }
        }
    }
}
client.indices.create(index=index_name, body=mappings)

# Sample document
doc = {
    "text": {
        "cve_id": "CVE-2024-12345",
        "description": "This is a test CVE vulnerability.",
        "severity": "High"
    }
}

# Index the document
response = client.index(index=index_name, document=doc)
print("Document indexed:", response)

Document indexed: {'_index': 'cve-index', '_id': 'wWBiX5cBOQ2nh5PuxeR9', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


POST /_security/api_key
{
  "name": "my-api-key",
  "role_descriptors": {
    "my-role": {
      "cluster": ["all"],
      "index": [
        {
          "names": ["cve-index"],
          "privileges": ["all"]
        }
      ]
    }
  }
}